<p style="font-family: Arial; font-size:3.75em; color:purple; font-style:bold">
<br>FANAL ANALYSIS VISUALIZATION<br></p>

This notebook illustrates the analysys phase made in FANALIC starting from the pseudo-reconstructed data performed also in FANALIC and dealing with the topology and the ROI stuff.
Following the "FANAL" way this is done in different steps:
> * Tracks building and filtering
> * Blobs  building and filtering
> * ROI filtering

It provides the contents of the dataframes generated together with some control plots associated.

In [ ]:
%matplotlib inline
%load_ext memory_profiler
%load_ext line_profiler

%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

#### Importing general stuff

In [ ]:
import os
import sys
import numpy  as np
import tables as tb
import pandas as pd
import matplotlib.pyplot as plt
from   matplotlib.colors import LogNorm

# Specific IC stuff
import invisible_cities.core.system_of_units  as units

# Specific fanalIC stuff
from fanal.reco.reco_io_functions import get_reco_group_name
from fanal.ana.ana_io_functions   import get_ana_group_name

from fanal.core.fanal_types  import DetName

#from fanal.mc.mc_utilities import print_mc_event
#from fanal.mc.mc_utilities import plot_mc_event

from fanal.fanal_ana       import fanal_ana

In [ ]:
plt.rcParams["figure.figsize"] = 8, 6
plt.rcParams["font.size"     ] = 14

# SETUP

In [ ]:
DET_NAME = "next100"
DET_NAME = getattr(DetName, DET_NAME)

ANA_FNAME = "../data/next100/bb0nu/ana/next100.bb0nu.fwhm_07.voxel_10x10x10.ana.h5"
#ANA_FNAME = "../data/next100/Bi214/ana/next100.Bi214.fwhm_07.voxel_10x10x10.ana.h5"
#ANA_FNAME = "../data/next100/Tl208/ana/next100.Tl208.fwhm_07.voxel_10x10x10.ana.h5"

#ANA_FNAME = "../data/next100/bb0nu/ana/next100.bb0nu.fwhm_05.voxel_3x3x3.ana.h5"
#ANA_FNAME = "../data/next100/Bi214/ana/next100.Bi214.fwhm_05.voxel_3x3x3.ana.h5"
#ANA_FNAME = "../data/next100/Tl208/ana/next100.Tl208.fwhm_05.voxel_3x3x3.ana.h5"

In [ ]:
with tb.open_file(ANA_FNAME, mode='r') as ana_file:
    ana_group_name  = '/FANAL/' + ana_file.root.FANAL.__members__[0]
    reco_group_name = ana_file.get_node_attr(ana_group_name, 'input_reco_group')
    event_type      = ana_file.get_node_attr(ana_group_name, 'event_type')
    roi_Emin        = ana_file.get_node_attr(ana_group_name, 'roi_Emin')
    roi_Emax        = ana_file.get_node_attr(ana_group_name, 'roi_Emax')

#### Verbosing the event counters

In [ ]:
with tb.open_file(ANA_FNAME, mode='r') as iFile:
    simulated_events     = iFile.get_node_attr(ana_group_name, 'simulated_events')
    stored_events        = iFile.get_node_attr(ana_group_name, 'stored_events')
    smE_filter_events    = iFile.get_node_attr(ana_group_name, 'smE_filter_events')
    fid_filter_events    = iFile.get_node_attr(ana_group_name, 'fid_filter_events')
    tracks_filter_events = iFile.get_node_attr(ana_group_name, 'tracks_filter_events')
    blobs_filter_events  = iFile.get_node_attr(ana_group_name, 'blobs_filter_events')
    roi_filter_events    = iFile.get_node_attr(ana_group_name, 'roi_filter_events')


print('* File Name : ', ANA_FNAME)
print('* Group Name: ', ana_group_name)
print('''* Event counters:
  Simulated events:     {:8}
  Stored events:        {:8}
  smE_filter events:    {:8}
  fid_filter events:    {:8}
  tracks_filter events: {:8}
  blobs_filter events:  {:8}
  roi_filter events:    {:8}'''
      .format(simulated_events, stored_events, smE_filter_events,
              fid_filter_events, tracks_filter_events,
              blobs_filter_events, roi_filter_events))


#### Verbosing the dataframes

In [ ]:
events_df = pd.read_hdf(ANA_FNAME, ana_group_name + '/events')
voxels_df = pd.read_hdf(ANA_FNAME, ana_group_name + '/voxels')

In [ ]:
events_df.head()

In [ ]:
voxels_df.head(10)

In [ ]:
events_df_smE_True    = events_df[events_df.smE_filter]
events_df_fid_True    = events_df[events_df.fid_filter]
events_df_tracks_True = events_df[events_df.tracks_filter]
events_df_blobs_True  = events_df[events_df.blobs_filter]
events_df_roi_True    = events_df[events_df.roi_filter]

In [ ]:
columns_toShow = ['smE', 'num_tracks', 'track0_E', 'track0_voxels', 'track0_length',
                  'tracks_filter', 'blob1_E', 'blob2_E', 'blobs_filter', 'roi_filter']
#events_df_fid_True[columns_toShow].head()
events_df_tracks_True[columns_toShow].head()
#events_df_blobs_True[columns_toShow].head()
#events_df_roi_True[columns_toShow].head()

#### Place for my quick & dirty checks

In [ ]:
#events_df_tracks_True.columns
columns_toShow = ['num_MChits', 'smE', 'smE_filter', 'num_voxels', 'voxel_sizeX', 'voxel_sizeY',
                  'voxel_sizeZ', 'voxels_minZ', 'voxels_maxZ', 'voxels_maxRad',
                  'veto_energy', 'fid_filter', 'num_tracks', 'blob1_E', 'blob2_E']
columns_toShow = ['smE', 'num_tracks', 'track0_E', 'track0_voxels', 'track0_length',
                  'tracks_filter', 'blob1_E', 'blob2_E', 'blobs_filter', 'roi_filter']
len(events_df_tracks_True[(events_df_tracks_True.track0_length < 30) 
                          | (events_df_tracks_True.track0_voxels <= 3)
                        ])
#events_df_tracks_True[(events_df_tracks_True.track0_length < 30) & events_df.roi_filter][columns_toShow]
#events_df_tracks_True[columns_toShow].describe().T
#events_df_tracks_True["track0_length"].hist(range=[0,100], bins=50)

#### Generating some histograms ...

In [ ]:
fig = plt.figure(figsize = (8,4))

counters_toPlot = {'smE_filter Events'   : smE_filter_events,
                   'fid_filter Events'   : fid_filter_events,
                   'tracks_filter Events': tracks_filter_events,
                   'blobs_filter Events' : blobs_filter_events,
                   'roi_filter Events'   : roi_filter_events}

plt.bar(range(len(counters_toPlot)), list(counters_toPlot.values()), align='center',
        color=['b', 'r', 'c', 'y', 'm'], yerr=np.sqrt([smE_filter_events, fid_filter_events,
                                                       tracks_filter_events, blobs_filter_events,
                                                       roi_filter_events]))
plt.xticks(range(len(counters_toPlot)), list(counters_toPlot.keys()), rotation='30')
plt.show()

Plotting number of tracks

In [ ]:
fig = plt.figure(figsize = (8,5))
num_bins = 10

plt.hist(events_df_fid_True.num_tracks, num_bins, [0, 10])
plt.xlabel('Num Tracks', size=12)
plt.ylabel('Num. events', size=12)
plt.title(f'{event_type} - Number of reconstructed tracks')

Plotting energies, number of voxels and lengths of the three hottest tracks

In [ ]:
fig = plt.figure(figsize = (18, 18))
num_E_bins = 20
num_voxels_bins = 20
num_length_bins = 20

# First track plots
plotting_events = events_df_fid_True[events_df_fid_True.num_tracks >= 1]
ax1 = fig.add_subplot(3, 3, 1)
plt.hist(plotting_events.track0_E, num_E_bins, [0., 2.5])
plt.title('Track 0 Energy [MeV]', size=12)

ax2 = fig.add_subplot(3, 3, 4)
plt.hist(plotting_events.track0_voxels, num_voxels_bins, [0, 100])
plt.title('Track 0 - Num Voxels', size=12)

ax3 = fig.add_subplot(3, 3, 7)
plt.hist(plotting_events.track0_length, num_length_bins, [0, 200])
plt.title('Track 0 - Length', size=12)

# Second track plots
plotting_events = events_df_fid_True[events_df_fid_True.num_tracks >= 2]
ax4 = fig.add_subplot(3, 3, 2)
plt.hist(plotting_events.track1_E, num_E_bins, [0., 1.0])
plt.title('Track 1 Energy [MeV]', size=12)

ax5 = fig.add_subplot(3, 3, 5)
plt.hist(plotting_events.track1_voxels, num_voxels_bins, [0, 20])
plt.title('Track 1 - Num Voxels', size=12)

ax6 = fig.add_subplot(3, 3, 8)
plt.hist(plotting_events.track1_length, num_length_bins, [0, 100])
plt.title('Track 1 - Length', size=12)

# Third track plots
plotting_events = events_df_fid_True[events_df_fid_True.num_tracks >= 3]
ax7 = fig.add_subplot(3, 3, 3)
plt.hist(plotting_events.track2_E, num_E_bins, [0., 1.0])
plt.title('Track 2 Energy [MeV]', size=12)

ax8 = fig.add_subplot(3, 3, 6)
plt.hist(plotting_events.track2_voxels, num_voxels_bins, [0, 20])
plt.title('Track 2 - Num Voxels', size=12)

ax9 = fig.add_subplot(3, 3, 9)
plt.hist(plotting_events.track2_length, num_length_bins, [0, 50])
plt.title('Track 2 - Length', size=12)


Energy histogram of events passing the tracks filter 

In [ ]:
fig = plt.figure(figsize = (15,5))
num_bins = 20

ax1 = fig.add_subplot(1, 2, 1)
plt.hist(events_df_tracks_True.track0_E, num_bins, [2.4, 2.5])
plt.xlabel('Track Energy [MeV]', size=14)
plt.title(f'{event_type} - Track Energy')

ax2 = fig.add_subplot(1, 2, 2)
plt.hist(events_df_tracks_True.smE, num_bins, [2.4, 2.5])
plt.xlabel('Event Energy [MeV]', size=14)
plt.title(f'{event_type} - Event Energy')

# As expected, both histograms are the same


Histograms of blob energies ...

In [ ]:
fig = plt.figure(figsize = (15,5))
num_bins = 30

ax1 = fig.add_subplot(1, 2, 1)
plt.hist(events_df_tracks_True.blob1_E, num_bins, [0, 1.5])
plt.xlabel('Blob1 Energy [MeV]')
plt.title(f'{event_type} - Blob1 Energy [MeV]', size=14)

ax2 = fig.add_subplot(1, 2, 2)
plt.hist(events_df_tracks_True.blob2_E, num_bins, [0, 1.5])
plt.xlabel('Blob2 Energy [MeV]')
plt.title(f'{event_type} - Blob2 Energy [MeV]', size=14)

fig = plt.figure(figsize = (7,6))
plt.hist2d(events_df_tracks_True.blob1_E, events_df_tracks_True.blob2_E, num_bins,
           [[0, 1.5], [0, 1.5]], norm=LogNorm())
plt.xlabel('Highest Blob Energy [MeV]')
plt.ylabel('Lowest Blob Energy [MeV]')
plt.title(f'{event_type} - Blobs Energies [MeV]', size=14)


ROI energy ...

In [ ]:
fig = plt.figure(figsize = (7,6))
num_bins = int((roi_Emax - roi_Emin) / units.keV)

plt.hist(events_df_roi_True.smE, num_bins, [roi_Emin, roi_Emax])
plt.xlabel('Event Energy [MeV]')
plt.title((f'{event_type} - Event Energy'), size=14)

In [ ]:
#voxels_df[voxels_df.negli].head()

In [ ]:
#columns_toShow = ['E', 'newE', 'track_id']
#voxels_df.loc[events_df_fid_True.index, columns_toShow]

In [ ]:
events_df_tracks_True[events_df_tracks_True.blob2_E < 0.05]

Printing and plotting the MC info of an event ...

In [ ]:
#event_to_print = 1311

#sim_files = []
#with tb.open_file(ANA_FILE_NAME, mode='r') as iFile:
#    reco_file_names = iFile.get_node_attr(ANA_GROUP_NAME, 'input_reco_files')
    
#    for reco_file_name in reco_file_names:
#        with tb.open_file(reco_file_name, mode='r') as iRecoFile:
#            sim_files += iRecoFile.get_node_attr(RECO_GROUP_NAME, 'input_sim_files')
            
#print_mc_event(event_to_print, sim_files, with_hits=True)

In [ ]:
#event_to_plot = 1311
#plot_mc_event(event_to_plot, sim_files)

In [ ]:
#voxels_df.loc[1311]